<a href="https://colab.research.google.com/github/AkshitAggarwal/TSAI_EVA5B2_Phase1/blob/main/Session_04/Assignment_Session04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils 
import torch.autograd
import torch.optim as optim
import torchvision.transforms as transforms

#Train Dataset and Train Dataloader

In [2]:
train_data = torchvision.datasets.EMNIST(
    root ='.\bin',
    download = True,
    train = True,
    split = 'byclass',
    transform = transforms.ToTensor()
)

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size = 32,
    shuffle = True,
)

Extracting .in/EMNIST/raw/emnist.zip to .in/EMNIST/raw
Processing byclass


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Processing bymerge
Processing balanced
Processing letters
Processing digits
Processing mnist
Done!


In [3]:
print(len(train_data), len(train_loader)) #Total no. of Training images and Training batches(Each batch has 32 images)

697932 21811


In [4]:
print(train_data.train_labels.bincount()) #Individual counts of all classes. Total classes: 62

tensor([34585, 38374, 34203, 35143, 33535, 31416, 34232, 35754, 33946, 33847,
         6407,  3878, 10094,  4562,  4934,  9182,  2517,  3152, 11946,  3762,
         2468,  5076,  9002,  8237, 24983,  8347,  2605,  5073, 20764,  9820,
        12602,  4637,  4695,  2771,  4743,  2701, 10033,  5159,  2854, 10177,
        24631,  2561,  3687,  8738,  2725,  1896,  2491, 15318,  2645, 11418,
         2749,  2448,  2994, 14105,  2699, 18262,  2830,  2910,  2697,  2822,
         2365,  2725])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [5]:
batch_iterator = next(iter(train_loader))
type(batch_iterator)

list

In [6]:
images, labels = batch_iterator
print(images.shape, labels.shape)

torch.Size([32, 1, 28, 28]) torch.Size([32])


In [7]:
sample = next(iter(train_data))
print(type(sample))
image, label = sample
print(image.shape, label)

<class 'tuple'>
torch.Size([1, 28, 28]) 35


#Network Architecture

In [23]:
class neural_net(nn.Module):
  def __init__(self):
    super(neural_net, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3) #insize= 1x28x28; outsize= 10x26x26; RF= 3
    self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3) #insize= 10x26x26; outsize= 10x24x24; RF= 5
    self.maxp1 = nn.MaxPool2d(kernel_size = 2, stride = 2) #insize= 10x24x24; outsize= 10x12x12; RF= 10
    self.conv3 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3) #insize= 10x12x12; outsize= 20x10x10; RF= 12
    self.conv4 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=3) #insize= 20x10x10; outsize= 20x8x8; RF= 14
    self.maxp2 = nn.MaxPool2d(kernel_size = 2, stride = 2) #insize= 20x8x8; outsize= 20x4x4; RF= 28
    self.conv5 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3) #insize= 20x4x4; outsize= 30x2x2; RF= 30
    self.outconv = nn.Conv2d(in_channels=30, out_channels=62, kernel_size=1) #insize = 30x2x2; outsize= 62x2x2; RF= 30
    self.avgpool = nn.AvgPool2d((2, 2)) #insize = 62x2x2; outsize= 62x1x1; RF=?
  
  def forward(self, t):
    t = self.maxp1(F.relu(self.conv2(F.relu(self.conv1(t)))))
    t = self.maxp2(F.relu(self.conv4(F.relu(self.conv3(t)))))
    t = self.outconv(F.relu(self.conv5(t)))
    t = self.avgpool(t)
    # t = t.view(-1, 62)
    return F.log_softmax(t, dim = 1)

In [24]:
network = neural_net()
print(network)

neural_net(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
  (maxp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1))
  (maxp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1))
  (outconv): Conv2d(30, 62, kernel_size=(1, 1), stride=(1, 1))
  (avgpool): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
)


In [25]:
torch.set_grad_enabled(False)

In [26]:
pred = network(image.unsqueeze(dim=0)) #Image is a rank-3 tensor, but our model expects a rank-4 'batch-like' tensor.
# print(pred.squeeze().shape) 

In [27]:
print(pred.shape)
# pred.eq(label)
print(pred.squeeze())

torch.Size([1, 62, 1, 1])
tensor([-4.1715, -4.0697, -3.9500, -3.9759, -4.1669, -4.0453, -4.1893, -4.2776,
        -3.9955, -4.0737, -4.1719, -4.2936, -4.1734, -4.2621, -4.1470, -4.2765,
        -4.0425, -3.9706, -4.0809, -4.1030, -4.0616, -4.2361, -4.0710, -4.2208,
        -4.1982, -4.1315, -4.1912, -4.2521, -4.1194, -4.1513, -3.9368, -3.9284,
        -4.1212, -3.9105, -4.1546, -3.9758, -3.9991, -4.2799, -4.2057, -4.1863,
        -4.2252, -4.0607, -4.1531, -4.2505, -4.2059, -3.9906, -4.2317, -3.9749,
        -4.2255, -4.2103, -4.1789, -4.1372, -4.2097, -3.9928, -3.9708, -4.2551,
        -4.2676, -4.2037, -4.2624, -4.0639, -4.1283, -4.2764])


Testing with a bunch of images. 

In [28]:
batch = next(iter(train_loader))
print(type(batch), len(batch))
images, labels = batch
print(images.shape, labels.shape)

<class 'list'> 2
torch.Size([32, 1, 28, 28]) torch.Size([32])


In [29]:
preds = network(images)
print(preds.shape)
preds = preds.argmax(dim=1)
print(preds.shape, preds)
print(preds.squeeze().eq(labels))

torch.Size([32, 62, 1, 1])
torch.Size([32, 1, 1]) tensor([[[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]],

        [[33]]])
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False])


In [30]:
# for param in network.parameters():
  # print(param.shape)
for name, param in network.named_parameters():
  print(name, '\t\t', param.shape)

conv1.weight 		 torch.Size([10, 1, 3, 3])
conv1.bias 		 torch.Size([10])
conv2.weight 		 torch.Size([10, 10, 3, 3])
conv2.bias 		 torch.Size([10])
conv3.weight 		 torch.Size([20, 10, 3, 3])
conv3.bias 		 torch.Size([20])
conv4.weight 		 torch.Size([20, 20, 3, 3])
conv4.bias 		 torch.Size([20])
conv5.weight 		 torch.Size([30, 20, 3, 3])
conv5.bias 		 torch.Size([30])
outconv.weight 		 torch.Size([62, 30, 1, 1])
outconv.bias 		 torch.Size([62])


#Shifting the model onto a GPU

In [31]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = neural_net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
            Conv2d-2           [-1, 10, 24, 24]             910
         MaxPool2d-3           [-1, 10, 12, 12]               0
            Conv2d-4           [-1, 20, 10, 10]           1,820
            Conv2d-5             [-1, 20, 8, 8]           3,620
         MaxPool2d-6             [-1, 20, 4, 4]               0
            Conv2d-7             [-1, 30, 2, 2]           5,430
            Conv2d-8             [-1, 62, 2, 2]           1,922
         AvgPool2d-9             [-1, 62, 1, 1]               0
Total params: 13,802
Trainable params: 13,802
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.14
Params size (MB): 0.05
Estimated Total Size (MB): 0.19
---------------------------------------------

#Training Phase

In [32]:
#Changing Autograd Universally back to True
torch.set_grad_enabled(True)

#Training Loop

In [33]:
def total_correct_predictions(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

#Defining training loop as a function for extra training.

The train function takes in as arguments:
1. Train Data Loader
2. The device on which the model is being trained on.
3. Optimizer(SGD in our case)
4. Total No. of predictions for Accuracy metric
5. No of epochs

In [34]:
def train_func(train_loader, device, optimizer, total_preds, epochs):
  for epoch in range(epochs):
  
    total_loss = 0
    total_correct = 0

    for batch in train_loader: #Iterate over all the batches
      images, labels = batch #Get images and labels from the batch
      images = images.to(device) #Load images onto the GPU
      labels = labels.to(device).reshape(-1, 1, 1) #Reshape labels tensor to fit the model      

      preds = model(images) #Get Predictions.
      loss = F.cross_entropy(preds, labels) #Calculate Loss. 

      optimizer.zero_grad() #Setting the gradient value to 0, so as to not accumulate it over and over.
      loss.backward()
      optimizer.step()

      total_loss += loss.item()
      total_correct += total_correct_predictions(preds, labels)

    acc = total_correct / total_preds
    print(
        "Epoch:",epoch,
        ", Loss:",total_loss,
        ", Total_correct_Predictions:",total_correct,
        ", Accuracy:",acc * 100
    )

In [38]:
# data_loader = utils.data.DataLoader(
    # train_data, 
    # batch_size = 128,
    # shuffle = True
# )

optimizer = optim.SGD(model.parameters(), lr = 0.1)

total_preds = len(train_data)
# 
# train_func(data_loader, device, optimizer, total_preds, 5)

In [39]:
data_loader = utils.data.DataLoader(train_data, batch_size = 256, shuffle = True)

train_func(data_loader, device, optimizer, total_preds, 20)

Epoch: 0 , Loss: 2807.6059895157814 , Total_correct_Predictions: 486977 , Accuracy: 69.77427600396601
Epoch: 1 , Loss: 1394.5725757479668 , Total_correct_Predictions: 574354 , Accuracy: 82.2936905027997
Epoch: 2 , Loss: 1293.2642947733402 , Total_correct_Predictions: 581681 , Accuracy: 83.34350624416132
Epoch: 3 , Loss: 1238.5196934640408 , Total_correct_Predictions: 585898 , Accuracy: 83.94771983517019
Epoch: 4 , Loss: 1202.541843086481 , Total_correct_Predictions: 588489 , Accuracy: 84.3189594401747
Epoch: 5 , Loss: 1178.7424369305372 , Total_correct_Predictions: 590105 , Accuracy: 84.55050062183709
Epoch: 6 , Loss: 1158.2486118674278 , Total_correct_Predictions: 591914 , Accuracy: 84.8096949272995
Epoch: 7 , Loss: 1142.4311785399914 , Total_correct_Predictions: 592930 , Accuracy: 84.95526784844368
Epoch: 8 , Loss: 1128.7998144179583 , Total_correct_Predictions: 594151 , Accuracy: 85.13021325859826
Epoch: 9 , Loss: 1117.230924665928 , Total_correct_Predictions: 594953 , Accuracy: 85.

In [40]:
optimizer = optim.SGD(model.parameters(), lr = 0.01) #Experimenting with larger LR due to extremely slow accuracy climb. 

train_func(data_loader, device, optimizer, total_preds, 10)

Epoch: 0 , Loss: 1013.139549061656 , Total_correct_Predictions: 603090 , Accuracy: 86.41099705988549
Epoch: 1 , Loss: 1009.9137730449438 , Total_correct_Predictions: 603532 , Accuracy: 86.47432701180058
Epoch: 2 , Loss: 1008.6723053455353 , Total_correct_Predictions: 603508 , Accuracy: 86.47088828137984
Epoch: 3 , Loss: 1007.8622366786003 , Total_correct_Predictions: 603563 , Accuracy: 86.47876870526069
Epoch: 4 , Loss: 1007.1349303871393 , Total_correct_Predictions: 603608 , Accuracy: 86.48521632479955
Epoch: 5 , Loss: 1006.4741647988558 , Total_correct_Predictions: 603707 , Accuracy: 86.49940108778506
Epoch: 6 , Loss: 1005.818158686161 , Total_correct_Predictions: 603734 , Accuracy: 86.50326965950838
Epoch: 7 , Loss: 1005.407583758235 , Total_correct_Predictions: 603680 , Accuracy: 86.49553251606173
Epoch: 8 , Loss: 1004.9339700341225 , Total_correct_Predictions: 603647 , Accuracy: 86.49080426173323
Epoch: 9 , Loss: 1004.4717433303595 , Total_correct_Predictions: 603866 , Accuracy: 8

In [43]:
torch.save(model.state_dict(), '/content/model_param_86')

In [42]:
!pwd

/content
